<div style="  color:#e3395e; padding: 15px; border-radius:5px;text-align: center; font-size: 30px">Nội dung</div>
    
[1. Đọc dữ liệu](#1) 
    
[2. Transform data](#2)
    
[3. Model](#3)
    
+    [3.1 Train](#3_1)
    
+    [3.2 Đánh giá](#3_2)
    
[4. Test](#4)
    
[5. Kết quả](#5)   
    
    

<a id="1"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">1. Đọc dữ liệu 📩📩</div>

In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import re
from transformers import AutoTokenizer ,BertForSequenceClassification, AdamW, RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf

In [51]:
# read train data
train_df=pd.read_csv("data/new_train.csv")
# read test data
test_df=pd.read_csv("data/new_test.csv")

In [52]:
train_df.head()

,text,label
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [53]:
test_df.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [54]:
train_df, valid_df = train_test_split(train_df, test_size = 0.15)

In [55]:
train_data=train_df.values.tolist()
val_data=valid_df.values.tolist()
test_data=test_df.values.tolist()

<a id="2"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">2. Transform data 🔄️</div>



In [56]:
class Mydata(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [57]:
train=Mydata(train_data)
valid=Mydata(val_data)

In [58]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [59]:
print("Model max_length: ", tokenizer.model_max_length)

Model max_length:  512


In [60]:
def collate_fn(data: list[tuple[str, int]]):
    texts = []
    labels = []
    for content, label in data:
        texts.append(content)
        labels.append(label)
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    return input_ids, labels

In [61]:
train_dataloader = torch.utils.data.DataLoader(dataset = train, batch_size = 32, collate_fn = collate_fn, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(dataset = valid, batch_size = 32, collate_fn = collate_fn)

In [62]:
for input_ids, labels in train_dataloader:
    print(input_ids, labels)
    break

tensor([[  101,  4268,  2024,  ...,     0,     0,     0],
        [  101, 19387,  1030,  ...,     0,     0,     0],
        [  101,  7867,  3764,  ...,     0,     0,     0],
        ...,
        [  101,  2066,  2005,  ...,     0,     0,     0],
        [  101,  2339,  2515,  ...,     0,     0,     0],
        [  101, 20005, 11138,  ...,     0,     0,     0]]) tensor([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 0])


In [63]:
torch.cuda.is_available()

True

<a id="3"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
3. Model 🤖🤖
</div>



In [64]:
class MyBertForClassification(torch.nn.Module):
    def __init__(self, num_labels) -> None:
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
    def forward(self, input_ids, labels):
        outputs = self.bert(input_ids=input_ids, labels=labels)
        probs = torch.softmax(outputs['logits'], dim = -1)
        preds = torch.argmax(probs, dim = -1)
        outputs['preds'] = preds
        return outputs

In [65]:
model = MyBertForClassification(2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [66]:
# Chuyển mô hình lên GPU
model.cuda()


MyBertForClassification(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_featu

<a id="3_1"></a>
<div style=" background-color:#dceff2 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">3.1 Train model ✍️✍️</div>



In [67]:
LR = 4e-6
EPOCH = 4
LOG_STEP = 30

In [68]:
optimizer = AdamW(model.parameters(), lr=LR)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [69]:
print(f"Train Loader Step: {len(train_dataloader)}")
print(f"Valid Loader Step: {len(valid_dataloader)}")

Train Loader Step: 203
Valid Loader Step: 36


In [70]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = []
precision = []
recall = []
f1 = []
loss_list = []

for epoch in range(EPOCH):
    running_loss = 0.0
    print(f"Epoch {epoch + 1}:")
    model.train()  # Chuyển sang chế độ huấn luyện
    for i, (input_id, label) in enumerate(train_dataloader):
        input_ids = input_id
        labels = label

        input_ids = input_ids.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(input_ids, labels)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % LOG_STEP == LOG_STEP-1:
            print('[Epoch %d, Batch %d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / LOG_STEP))
            running_loss = 0.0

    model.eval()
    valid_loss = 0
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for step, (input_ids, labels) in enumerate(valid_dataloader):
            input_ids = input_ids.cuda()
            labels = labels.cuda()
            outputs = model(input_ids, labels)
            loss = outputs['loss']
            valid_loss += loss.item()


            true_labels.extend(labels.tolist())
            pred_labels.extend(outputs['preds'].tolist())

    accuracy.append(accuracy_score(true_labels, pred_labels))
    precision.append(precision_score(true_labels, pred_labels, average='macro'))
    recall.append(recall_score(true_labels, pred_labels, average='macro'))
    f1.append(f1_score(true_labels, pred_labels, average='macro'))
    loss_list.append((valid_loss / len(valid_dataloader)))
    print(f"Valid loss: {valid_loss / len(valid_dataloader)}")

# Lưu mô hình
torch.save(model.state_dict(), 'bert_classifier.pth')
# model.load_state_dict(torch.load('bert_classifier.pth'))
# model.eval()

Epoch 1:
[Epoch 1, Batch 30] loss: 0.680
[Epoch 1, Batch 60] loss: 0.625
[Epoch 1, Batch 90] loss: 0.594
[Epoch 1, Batch 120] loss: 0.513
[Epoch 1, Batch 150] loss: 0.494
[Epoch 1, Batch 180] loss: 0.491
Valid loss: 0.4072616795698802
Epoch 2:
[Epoch 2, Batch 30] loss: 0.432
[Epoch 2, Batch 60] loss: 0.421
[Epoch 2, Batch 90] loss: 0.406
[Epoch 2, Batch 120] loss: 0.425
[Epoch 2, Batch 150] loss: 0.389
[Epoch 2, Batch 180] loss: 0.413
Valid loss: 0.3784695081412792
Epoch 3:
[Epoch 3, Batch 30] loss: 0.384
[Epoch 3, Batch 60] loss: 0.346
[Epoch 3, Batch 90] loss: 0.316
[Epoch 3, Batch 120] loss: 0.365
[Epoch 3, Batch 150] loss: 0.408
[Epoch 3, Batch 180] loss: 0.384
Valid loss: 0.35993850355346996
Epoch 4:
[Epoch 4, Batch 30] loss: 0.370
[Epoch 4, Batch 60] loss: 0.340
[Epoch 4, Batch 90] loss: 0.337
[Epoch 4, Batch 120] loss: 0.313
[Epoch 4, Batch 150] loss: 0.341
[Epoch 4, Batch 180] loss: 0.331
Valid loss: 0.36693356393112075


<a id="3_2"></a>
<div style=" background-color:#dceff2 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">
3.2 Đánh giá mô hình 👍👍</div>



In [71]:
df_score = pd.DataFrame({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Loss': loss_list }, index=
            ['Epoch 1','Epoch 2','Epoch 3', 'Epoch 4'])

In [72]:
df_score

,Accuracy,Precision,Recall,F1,Loss
Epoch 1,0.836252,0.843957,0.823688,0.829140,0.407262
Epoch 2,0.840630,0.849418,0.827789,0.833521,0.378470
Epoch 3,0.855517,0.861525,0.844961,0.850000,0.359939
Epoch 4,0.853765,0.854478,0.846738,0.849647,0.366934


Hiệu suất của mô hình được cải thiện qua mỗi epoch trong đó `Accuracy`,	`Precision`, `Recall`, `F1` tăng dần và `Loss` giảm dần

## Model 2

### Split data

In [73]:
train_df

,text,label
5809,@fa07af174a71408 I have lived &amp; my family ...,1
1217,Bradford. Back to doing what we do best. Burni...,1
6847,#NissanNews : Trauma Alert and 1 Child Among 6...,1
6194,It's 'Run From Sirens' by 'Half Hour Hotel' @h...,0
1497,Learning from the Legacy of a Catastrophic Eru...,1
...,...,...
7372,Windstorm board OKs rate hike before change ht...,0
6708,Severe Thunderstorm Warning for Oklahoma Count...,1
5621,./.....hmm 12000 Nigerian refugees repatriated...,1
3309,myGC: Broken powerlines evacuate Gold Coast tr...,1


In [75]:
X_train = train_df['text'].to_numpy()
y_train = train_df['label'].to_numpy()
# X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.15, stratify = y_full)
X_val = valid_df['text'].to_numpy()
y_val = valid_df['label'].to_numpy()

In [141]:
y_train

array([1, 1, 1, ..., 1, 1, 1])

In [76]:
X_test = test_df['text'].to_list()

### Tokenizer

In [77]:
tokenizer_roberta = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)

In [88]:
def get_tokenizer_max_len(data):
    token_lens = []
    for txt in data:
        tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
        token_lens.append(len(tokens))
    max_length=np.max(token_lens)
    return max_length

In [168]:
def tokenizer_fun(data, max_len=MAX_LEN) : #
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

In [169]:
def get_tokenized_ds(X_data, y_data, tokenizer, batch_size=32, prefetch=True):
    X_data_tokenized = tokenizer(X_data)
    X_data_tokenized_ds = tf.data.Dataset.from_tensor_slices(X_data_tokenized)
    if len(y_data) > 0:
        y_data_ds = tf.data.Dataset.from_tensor_slices(y_data)
        data_ds = tf.data.Dataset.zip((X_data_tokenized_ds, y_data_ds))
    else:
        data_ds = tf.data.Dataset.from_tensor_slices((X_data_tokenized,))
    if batch_size>0:
        data_ds = data_ds.batch(batch_size)
    if prefetch:
        data_ds = data_ds.prefetch(tf.data.AUTOTUNE)
    return data_ds

In [170]:
MAX_LEN = get_tokenizer_max_len(X_train) if get_tokenizer_max_len(X_train) > get_tokenizer_max_len(X_val) else get_tokenizer_max_len(X_val)
MAX_LEN 

91

In [171]:
train_roberta_ds = get_tokenized_ds(X_train, y_train, tokenizer_fun)
val_roberta_ds = get_tokenized_ds(X_val, y_val, tokenizer_fun)
test_roberta_ds = get_tokenized_ds(X_test, [], tokenizer_fun, prefetch=False)
train_roberta_ds.element_spec, val_roberta_ds.element_spec, test_roberta_ds.element_spec

(((TensorSpec(shape=(None, 91), dtype=tf.int64, name=None),
   TensorSpec(shape=(None, 91), dtype=tf.int64, name=None)),
  TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
 ((TensorSpec(shape=(None, 91), dtype=tf.int64, name=None),
   TensorSpec(shape=(None, 91), dtype=tf.int64, name=None)),
  TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
 ((TensorSpec(shape=(None, 91), dtype=tf.int64, name=None),
   TensorSpec(shape=(None, 91), dtype=tf.int64, name=None)),))

### Train

In [172]:
model_roberta = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [173]:
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [174]:
%%time
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(
    patience=2, monitor="val_accuracy", factor=0.3, verbose=1)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=3, restore_best_weights=True)
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
model_roberta.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=opt, metrics=["accuracy"])
model_roberta.fit(
        train_roberta_ds,
        validation_data=val_roberta_ds,
        epochs=10,
        callbacks=[early_stopping_cb, reduce_lr_cb])

Epoch 1/5
203/203 [==============================] - 139s 436ms/step - loss: 0.4722 - accuracy: 0.7727 - val_loss: 0.3572 - val_accuracy: 0.8625
Epoch 2/5
203/203 [==============================] - 77s 381ms/step - loss: 0.3578 - accuracy: 0.8490 - val_loss: 0.3679 - val_accuracy: 0.8529
Epoch 3/5
203/203 [==============================] - 77s 380ms/step - loss: 0.2974 - accuracy: 0.8822 - val_loss: 0.4157 - val_accuracy: 0.8354
Epoch 4/5
203/203 [==============================] - 77s 381ms/step - loss: 0.2553 - accuracy: 0.9019 - val_loss: 0.4660 - val_accuracy: 0.8266
Epoch 5/5
203/203 [==============================] - 77s 380ms/step - loss: 0.2134 - accuracy: 0.9170 - val_loss: 0.4350 - val_accuracy: 0.8398
CPU times: user 5min 42s, sys: 30.2 s, total: 6min 12s
Wall time: 7min 32s


### Validation

In [191]:
y_val_predictions = model_roberta.predict(val_roberta_ds)

36/36 [==============================] - 8s 131ms/step


In [192]:
pd.Series(np.argmax(y_val_predictions.logits, axis=1) == y_val).value_counts(normalize=True)

True     0.839755
False    0.160245
dtype: float64

## Test

In [175]:
y_predictions = model_roberta.predict(test_roberta_ds)

102/102 [==============================] - 17s 135ms/step


In [177]:
from scipy.special import softmax

In [178]:
result = pd.DataFrame()
result['id'] = test_df[['id']].copy()
arr = softmax(y_predictions.logits, axis=-1)
arr

array([[0.13291585, 0.86708415],
       [0.00406606, 0.99593395],
       [0.01038038, 0.9896197 ],
       ...,
       [0.00180637, 0.9981937 ],
       [0.05505515, 0.94494486],
       [0.00644211, 0.9935579 ]], dtype=float32)

In [179]:
new_arr = np.where(arr[:, 0] > arr[:, 1], 0, 1)
result['target'] = new_arr

In [180]:
df_crr = pd.read_csv('/kaggle/input/new-tweet/submission.csv')

In [181]:
(result['target'] == df_crr['target']).value_counts(normalize=True)

True     0.821637
False    0.178363
Name: target, dtype: float64

<a id="4"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
4. Test 🔍🔍
</div>

In [182]:
def collate_fn(data: list[tuple[str, int]]):
    texts = []
    ids = []
    for id, content in data:
        texts.append(content)
        ids.append(id)
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    ids = torch.tensor(ids)
    return input_ids, ids

In [183]:
inputs,ids=collate_fn(test_data)

In [184]:
test=Mydata(test_data)

In [185]:
test_dataloader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 1, collate_fn = collate_fn)

In [186]:
result_test={
    'id':[],
    'target':[]
}

In [187]:
with torch.no_grad():
    for step, (input_ids, idx) in enumerate(test_dataloader):
        input_ids = input_ids.cuda()
        # labels = labels.cuda()
        outputs = model(input_ids=input_ids, labels=None)
        preds = outputs['preds']
        span = preds[0].item()
        result_test['id'].append(idx.item())
        result_test['target'].append(span)

In [188]:
df=pd.DataFrame(result_test)

In [189]:
df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [190]:
df.to_csv("result1.csv",index=False)

<a id="5"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
5. Kết quả 📜📜
</div>



![result](result.png)  